In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
# read data
top_dir = '/Users/stiso/Documents/Python/graphLearning/ECoG data/behavioral_data_raw/'
save_dir = '/Users/stiso/Documents/Python/graphLearning/ECoG data/behavior_preprocessed/'
subj = [1,2,4,6]

In [3]:
#%% Load and concatenate data

data = pd.DataFrame()
for i in subj:
    curr = pd.read_csv("".join([top_dir, 'subj', str(i), '/subj', str(i), '_clean_data']))
    curr['subj'] = i
    data = data.append(curr)
    

/Users/stiso/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [4]:
data['subj'].unique()

array([1, 2, 4, 6])

In [10]:
data.head(20)

,ISI_raw,Unnamed: 0,block,correct_raw,graph,hand,hand_transition,lag10,onset_raw,order,...,recency,resp,resp_raw,rt_raw,sess,subj,transition,trialNum,typing_raw,walk
0,0.05,5,1.0,1.0,lattice,left,False,1,7.087798,6.0,...,2.0,q,['q'],0.945559,1.0,1,False,6.0,a,2.0
1,0.05,8,1.0,1.0,lattice,right,False,1,10.337893,9.0,...,2.0,b,['b'],0.752210,1.0,1,False,9.0,a,0.0
2,0.05,9,1.0,1.0,lattice,right,False,1,11.187585,10.0,...,2.0,u,['u'],0.581672,1.0,1,False,10.0,a,8.0
3,0.05,12,1.0,1.0,lattice,right,True,0,13.455660,13.0,...,11.0,p,['p'],0.810152,1.0,1,False,13.0,a,6.0
4,0.05,13,1.0,1.0,lattice,right,False,2,14.357635,14.0,...,4.0,u,['u'],1.003771,1.0,1,False,14.0,a,8.0
5,0.05,14,1.0,1.0,lattice,right,False,2,15.457186,15.0,...,6.0,b,['b'],0.735995,1.0,1,False,15.0,a,0.0
6,0.05,15,1.0,1.0,lattice,right,False,3,16.288177,16.0,...,2.0,u,['u'],0.808980,1.0,1,False,16.0,a,8.0
7,0.05,16,1.0,1.0,lattice,left,True,1,17.190220,17.0,...,5.0,e,['e'],1.387098,1.0,1,False,17.0,a,7.0
8,0.05,17,1.0,1.0,lattice,left,False,1,18.674009,18.0,...,7.0,r,['r'],0.799790,1.0,1,False,18.0,a,9.0
9,0.05,18,1.0,1.0,lattice,left,False,0,19.574129,19.0,...,14.0,v,['v'],1.122980,1.0,1,False,19.0,a,1.0


In [9]:
# add session for subjects that dont have it
data.loc[data['sess'].isna(),'sess'] = 1

In [12]:
data.describe()

,ISI_raw,Unnamed: 0,block,correct_raw,lag10,onset_raw,order,pID,recency,rt_raw,sess,subj,trialNum,walk
count,4671.00,4671.000000,4671.000000,4671.000000,4671.000000,4671.000000,4671.000000,4671.000000,4671.000000,4671.000000,4671.000000,4671.000000,4671.000000,4671.000000
mean,0.05,498.667309,2.492186,0.988011,1.260972,511.191747,499.667309,3.839863,9.636266,0.732159,1.204239,3.839863,499.667309,4.377649
std,0.00,285.000200,1.110922,0.108847,1.048183,343.705256,285.000200,2.037350,11.723702,0.283757,0.403188,2.037350,285.000200,2.902256
min,0.05,1.000000,1.000000,0.000000,0.000000,4.096814,2.000000,1.000000,2.000000,0.283854,1.000000,1.000000,2.000000,0.000000
25%,0.05,251.000000,2.000000,1.000000,0.000000,226.961533,252.000000,2.000000,2.000000,0.559546,1.000000,2.000000,252.000000,2.000000
50%,0.05,498.000000,2.000000,1.000000,1.000000,488.988586,499.000000,4.000000,5.000000,0.672163,1.000000,4.000000,499.000000,4.000000
75%,0.05,742.500000,3.000000,1.000000,2.000000,713.043243,743.500000,6.000000,12.000000,0.819452,1.000000,6.000000,743.500000,7.000000
max,0.05,999.000000,4.000000,1.000000,5.000000,1587.839111,1000.000000,6.000000,126.000000,5.751362,2.000000,6.000000,1000.000000,9.000000


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4671 entries, 0 to 1911
Data columns (total 22 columns):
ISI_raw            4671 non-null float64
Unnamed: 0         4671 non-null int64
block              4671 non-null float64
correct_raw        4671 non-null float64
graph              4671 non-null object
hand               4671 non-null object
hand_transition    4671 non-null bool
lag10              4671 non-null int64
onset_raw          4671 non-null float64
order              4671 non-null float64
pID                4671 non-null float64
path               4671 non-null object
recency            4671 non-null float64
resp               4671 non-null object
resp_raw           4671 non-null object
rt_raw             4671 non-null float64
sess               4671 non-null float64
subj               4671 non-null int64
transition         4671 non-null bool
trialNum           4671 non-null float64
typing_raw         4671 non-null object
walk               4671 non-null float64
dtypes: b

In [14]:
data.to_csv("".join([save_dir, 'group_behavior.csv']))
